# Code bits used in the documentation for Andante

## Parser example from section 1.3

In [1]:
from andante.parser import Parser
parser = Parser()
text = """
:- begin_bg.
    mortal(X):-man(X).
    man(socrates).
:- end_bg.
"""
knowledge = parser.parse(text, rule='background')
print(knowledge)

Knowledge object (class: TreeShapedKnowledge)
Clauses:
   mortal(X) :- man(X).
   man(socrates).


## Solver example from section 2.2

In [2]:
from andante.solver import AndanteSolver
solver = AndanteSolver()
q = parser.parse("mortal(X).", rule="query")

# Using method succeeds_on
success = solver.succeeds_on(q, knowledge)
if success:
    print("The query succeeded")
else:
    print("The query failed")
    
# Using method query
print("The solutions for the query are:")
for solution in solver.query(q, knowledge):
    print('\t', solution)

The query succeeded
The solutions for the query are:
	 {X: socrates}


## Learner example from section 3.3

In [3]:
text = """
% Mode declarations
modeh(*,daughter(+person,-person)).
modeb(*,parent(+person,-person)).
modeb(*,parent(-person,+person)).
modeb(*,female(+person)).
modeb(*,female(-person)).

% Determinations
determination(daughter/2,parent/2).
determination(daughter/2,female/1).

% Background knowledge
:- begin_bg.
person(ann). person(mary). person(tom). person(eve). person(lucy).
female(ann). female(mary). female(eve). female(lucy).
parent(ann,mary).
parent(ann,tom).
parent(tom,eve).
parent(tom,lucy).
:- end_bg.
    
% Positive examples
:- begin_in_pos.
daughter(lucy,tom).
daughter(mary,ann).
daughter(eve,tom).
:- end_in_pos.

% Negative examples
:- begin_in_neg.
daughter(tom,ann).
daughter(tom,eve).
:- end_in_neg.
"""
pr = parser.parse(text, rule="andantefile")
H = pr.induce()

print("Knowledge learned")
print(H)

Knowledge learned
Knowledge object (class: TreeShapedKnowledge)
Clauses:
   daughter(A, B) :- female(A).


## Interface example from section 4

In [4]:
from andante.program import AndanteProgram
from andante.interface import MainInterface

ap = AndanteProgram.build_from("Examples/family.pl")
interface = MainInterface(ap)
interface.widget